In [ ]:
%run "../../../setup.ipynb"
if CLIENT == None:
    print("Failed to create CLIENT.")

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain import SerpAPIWrapper
from langchain.vectorstores import Weaviate
from typing import List
import os
import weaviate

def openai_chat_client():
  deployment = os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT_ID")
  key = os.getenv("AZURE_OPENAI_API_KEY")
  resource = os.getenv("AZURE_OPENAI_RESOURCE_NAME")

  return AzureOpenAI(deployment_name=deployment, 
                     openai_api_key=key, 
                     openai_api_base=f"https://{resource}.openai.azure.com/",
                     openai_api_version="2023-03-15-preview",
                     temperature=0.0)


def weaviate_vector_store(index_name: str, text_key: str, attributes: List[str] = []):
  weaviate_url = os.environ['WEAVIATE_URL']
  weaviate_api_key = os.environ['WEAVIATE_API_KEY']
  openai_api_key = os.environ['AZURE_OPENAI_API_KEY']

  auth_config = weaviate.AuthApiKey(api_key=weaviate_api_key)

  client = weaviate.Client(
      url=weaviate_url,
      auth_client_secret=auth_config
  )
  return Weaviate(client=client, 
                  index_name=index_name, 
                  text_key=text_key, 
                  attributes=attributes)

In [ ]:
index_name = "Work"
text_key = "title"
attributes = ["title", "source", "alternate_title", "contributor", "create_date", "creator", "date_created", "description", "genre", "keywords", "language", "location", "physical_description_material", "physical_description_size", "scope_and_contents", "style_period", "subject", "table_of_contents", "technique"]

weaviate = weaviate_vector_store(
  index_name=index_name, 
  text_key=text_key, 
  attributes=attributes
)

client = openai_chat_client()

In [ ]:
query = "postcards of Northwestern University"
docs = weaviate_vector_store(
  index_name=index_name, 
  text_key=text_key, 
  attributes=attributes
).similarity_search_by_text(query, k=10, additional=["id", "certainty"])
docs

In [ ]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever
r = WeaviateHybridSearchRetriever(weaviate.Client(
      url=weaviate_url,
      auth_client_secret=auth_config,
      additional_headers={
          "X-OpenAI-Api-Key": openai_api_key
      }
  ), index_name=index_name, text_key=text_key, attributes=attributes, k=8)
r.get_relevant_documents("west africa")

In [ ]:
import base64
import json
import os
from langchain.chains import RetrievalQAWithSourcesChain


def ask(question):
  chain = RetrievalQAWithSourcesChain.from_chain_type(
    client, 
    chain_type="stuff", 
    retriever=weaviate.as_retriever()
  )

  return chain({"question": question})

In [ ]:
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo,
)

print(client)

def agent(question):
    vectorstore_info = VectorStoreInfo(
        name=index_name,
        description="NULDC Works",
        vectorstore=weaviate
    )
    toolkit = VectorStoreToolkit(vectorstore_info=vectorstore_info, llm=client)
    agent_executor = create_vectorstore_agent(llm=client, toolkit=toolkit, verbose=True)
    return agent_executor.run(question)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores.base import VectorStoreRetriever
retrievalQA = RetrievalQA.from_llm(llm=client, retriever=weaviate.as_retriever(), return_source_documents=True)
res = retrievalQA.run(input_documents=docs, query="Does the Winterton Collection contain photos of Kenya?")
jprint(res)
# chain = load_qa_with_sources_chain(client, chain_type="stuff")
# query = "football stadium"
# docs = weaviate.similarity_search_by_text(query, k=10)
# jprint(chain({"input_documents": docs, "question": "What is the name of the Northwestern football stadium?"}))

In [ ]:
agent("Who were some headliners of the Berkeley Folk Music Festival?")

In [ ]:
ask("Where is Northwestern University located?")

In [ ]:
ask("Is Northwestern near Lake Michigan?")

In [ ]:
ask("Does the collection have any images of lions in the wild?")

In [ ]:
ask("Can you show me examples of war propaganda?")

In [ ]:
ask("Can you show me historical Northwestern football matches?")

In [ ]:
ask("Who did John Cage correspond with during his life?")

In [ ]:
ask("Clarinet is my favorite, any examples?")